# Ejemplo de Uso de config.py

Este notebook muestra cómo usar el archivo `config.py` para gestionar rutas automáticamente sin hardcodear usuarios.

## 1. Importar config.py (sin hardcodear rutas)

Este código detecta automáticamente la raíz del proyecto buscando el archivo `config.py`:

In [ ]:
import sys
from pathlib import Path

# Detectar automáticamente la raíz del proyecto
project_root = Path.cwd()
while not (project_root / 'config.py').exists() and project_root != project_root.parent:
    project_root = project_root.parent

# Agregar al path si no está ya
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✅ Raíz del proyecto: {project_root}")

# Importar configuración
import config

print(f"✅ Usuario detectado: {config.CURRENT_USER}")

## 2. Ver información de configuración

In [ ]:
# Mostrar información del sistema
config.info()

## 3. Usar rutas de constantes

In [ ]:
# Rutas de constantes (igual para todos)
R_Ips = config.IPS_CODIGO
R_Departamentos = config.DEPARTAMENTOS

print(f"IPS_CODIGO: {R_Ips}")
print(f"Existe: {R_Ips.exists()}")

## 4. Configurar rutas para una fecha específica

### Opción A: Usar función helper (Recomendado)

In [ ]:
# Configurar todas las rutas para una fecha
rutas = config.configurar_rutas_fecha(2026, 1, 16)

# Acceder a las rutas
R_Ms_ADRES_EPSC25 = rutas['maestro_contributivo']
R_Ms_ADRES_EPS025 = rutas['maestro_subsidiado']
R_NS_NEG = rutas['ns_negado']
R_NS_SIE = rutas['ns_sie']
R_NS_No_Enviar = rutas['ns_no_enviar']
R_NS_Enviar = rutas['ns_enviar']
R_Ips = rutas['ips_codigo']
R_Salida = rutas['salida']

print("Rutas configuradas para 16-01-2026:")
print(f"Maestro EPSC25: {R_Ms_ADRES_EPSC25}")
print(f"Maestro EPS025: {R_Ms_ADRES_EPS025}")
print(f"NS Negado: {R_NS_NEG}")
print(f"NS SIE: {R_NS_SIE}")
print(f"Salida: {R_Salida}")

### Opción B: Usar funciones individuales

In [ ]:
# Configurar rutas individualmente
year, month, day = 2026, 1, 16

R_Ms_ADRES_EPSC25 = config.get_maestro_contributivo(year, month, day)
R_Ms_ADRES_EPS025 = config.get_maestro_subsidiado(year, month, day)
R_NS_NEG = config.get_ns_negado(year, month, day)
R_NS_SIE = config.get_ns_sie(year, month, day)
R_NS_No_Enviar = config.get_ns_no_enviar(year, month, day)
R_NS_Enviar = config.get_ns_enviar(year, month, day)
R_Ips = config.IPS_CODIGO
R_Salida = config.get_traslados_output_dir(year, month, day)

print(f"Salida: {R_Salida}")

## 5. Crear directorio de salida si no existe

In [ ]:
# Asegurar que el directorio de salida existe
R_Salida_creado = config.ensure_dir(R_Salida)
print(f"✅ Directorio asegurado: {R_Salida_creado}")

## 6. Ejemplo de flujo completo

In [ ]:
import pandas as pd

# Configurar fecha de trabajo
FECHA_TRABAJO = (2026, 1, 16)  # año, mes, día

# Obtener todas las rutas
rutas = config.configurar_rutas_fecha(*FECHA_TRABAJO)

# Cargar archivos
try:
    # Leer IPS
    if rutas['ips_codigo'].exists():
        df_ips = pd.read_csv(rutas['ips_codigo'], sep='|', encoding='latin1')
        print(f"✅ IPS cargado: {len(df_ips)} registros")
    
    # Verificar maestros
    if rutas['maestro_contributivo'].exists():
        print(f"✅ Maestro EPSC25 encontrado")
    else:
        print(f"⚠️  Maestro EPSC25 no encontrado en: {rutas['maestro_contributivo']}")
    
    # Asegurar directorio de salida
    config.ensure_dir(rutas['salida'])
    print(f"✅ Directorio de salida listo: {rutas['salida']}")
    
except Exception as e:
    print(f"❌ Error: {e}")

## 7. Ventajas de este enfoque

✅ **No más hardcodeo de rutas** - Todo se detecta automáticamente  
✅ **Funciona en cualquier PC** - Solo necesita que exista la estructura de OneDrive  
✅ **Fácil de cambiar fechas** - Solo cambias 3 números (año, mes, día)  
✅ **Sin conflictos en Git** - El mismo código funciona para todos  
✅ **Mantenible** - Cambios en estructura se hacen en un solo lugar  